In [ ]:
import pandas as pd
import yfinance as yf
import requests
import pytz
import time
from datetime import datetime

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
"""
# Ativos
wallet = pd.DataFrame([
    {"Ticker": "B3SA3.SA", "Quantidade": 50, "Preço Médio": 9.93, "Data de Compra": "2024-12-09"},
    {"Ticker": "BTCI11.SA", "Quantidade": 36, "Preço Médio": 8.11, "Data de Compra": "2024-12-09"},
    {"Ticker": "BTHF11.SA", "Quantidade": 5, "Preço Médio": 8.73, "Data de Compra": "2025-07-01"},
    {"Ticker": "ROXO34.SA", "Quantidade": 41, "Preço Médio": 12.15, "Data de Compra": "2024-12-09"},
    {"Ticker": "bitcoin", "Quantidade": 0.00005449, "Preço Médio": 593407.91, "Data de Compra": "2025-06-28"}
])
"""

wallet = pd.DataFrame([
    {"Ticker": "bitcoin", "Quantidade": 0.00005449, "Preço Médio": 593407.91, "Data de Compra": "2025-06-28"}
])

# Cria novas colunas para os resultados
wallet['Cotação Atual'] = None
wallet['Valor Atual'] = None
wallet['Valor Investido'] = wallet['Quantidade'] * wallet['Preço Médio']
wallet['Lucro/Prejuízo (R$)'] = None
wallet['Rentabilidade Total (%)'] = None
wallet['Rentabilidade Anualizada (%)'] = None

# Função para pegar Cotação Atual de criptomoedas
br_timezone = pytz.timezone("America/Sao_Paulo")
hoje_brasilia = datetime.now(br_timezone)

def get_crypto_price_brl(crypto_ids):
    """
    Retorna o preço em BRL de uma ou mais criptomoedas usando a CoinGecko API.
    
    Parâmetro:
        crypto_ids (str ou list): Nome(s) no padrão CoinGecko (ex: 'bitcoin', 'ethereum', 'solana')
    
    Retorna:
        dict: {'bitcoin': 342000.0, 'ethereum': 19000.0}
    """
    # Converte string para lista
    if isinstance(crypto_ids, str): 
        crypto_ids = [crypto_ids]

    # Cria uma string separada por vírgulas
    ids_str = ','.join(crypto_ids)
    url = f"https://api.coingecko.com/api/v3/simple/price?ids={ids_str}&vs_currencies=brl"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        raise ValueError(f"Erro ao buscar preço das criptomoedas: {crypto_ids}")

# Processa cada ativo
for i, row in wallet.iterrows():
    ticker = row['Ticker']
    quantidade = row['Quantidade']
    preco_medio = row['Preço Médio']
    data_compra = pd.to_datetime(row['Data de Compra']).tz_localize('America/Sao_Paulo')

    # Dias em posse
    dias_posse = (hoje_brasilia - data_compra).days
    if dias_posse <= 0:
        dias_posse = 1  # Evita divisão por zero

    try:
        # Tenta pegar com yfinance (ativo tradicional)
        dados = yf.download(ticker, period='1d', interval='1m', progress=False, auto_adjust=True)
        preco_atual = float(dados['Close'].dropna().iloc[-1].squeeze())
    except Exception as e:
        print(f"[AVISO] Não foi possível obter {ticker} via yfinance. Tentando CoinGecko...")

        try:
            preco_atual = get_crypto_price_brl(ticker)[ticker]['brl']
            time.sleep(10)
        except Exception as cg_err:
            print(f"[ERRO] Falha também com CoinGecko para {ticker}: {cg_err}")
            preco_atual = None

    valor_atual = quantidade * preco_atual
    valor_investido = quantidade * preco_medio
    lucro = valor_atual - valor_investido
    rent_total = (valor_atual / valor_investido) - 1
    rent_anual = (1 + rent_total) ** (365 / dias_posse) - 1

    wallet.at[i, 'Cotação Atual'] = round(preco_atual, 2)
    wallet.at[i, 'Valor Atual'] = round(valor_atual, 2)
    wallet.at[i, 'Lucro/Prejuízo (R$)'] = round(lucro, 2)
    wallet.at[i, 'Rentabilidade Total (%)'] = round(rent_total * 100, 2)
    wallet.at[i, 'Rentabilidade Anualizada (%)'] = round(rent_anual * 100, 2)

# Exibe resultado
wallet


1 Failed download:
['BITCOIN']: YFPricesMissingError('possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")')


[AVISO] Não foi possível obter bitcoin via yfinance. Tentando CoinGecko...


,Ticker,Quantidade,Preço Médio,Data de Compra,Cotação Atual,Valor Atual,Valor Investido,Lucro/Prejuízo (R$),Rentabilidade Total (%),Rentabilidade Anualizada (%)
0,bitcoin,0.000054,593407.91,2025-06-28,582931,31.76,32.334797,-0.57,-1.77,-66.16


In [ ]:
cache_precos = {}
cache_precos_realtime_cripto = {}

def get_crypto_price_brl_on_date_cached(crypto_id: str, date: str) -> float:
    chave = f"{crypto_id}_{date}"
    
    # Retorna do cache se já buscado
    if chave in cache_precos:
        return cache_precos[chave]

    # Caso contrário, busca na API
    try:
        date_obj = datetime.strptime(date, "%Y-%m-%d")
        formatted_date = date_obj.strftime("%d-%m-%Y")
    except ValueError:
        raise ValueError("Data deve estar no formato YYYY-MM-DD")

    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/history?date={formatted_date}&localization=false"
    response = requests.get(url)

    if response.status_code != 200:
        raise ValueError(f"Erro ao buscar histórico de {crypto_id} em {date} (status {response.status_code})")

    data = response.json()

    try:
        preco = data["market_data"]["current_price"]["brl"]
        cache_precos[chave] = preco
        time.sleep(1)  # espera para não sobrecarregar a API
        return preco
    except KeyError:
        raise ValueError(f"Preço em BRL não disponível para {crypto_id} em {date}")

# Função para rentabilidade de um ativo individual
def calcular_rentabilidade_ativo(ticker, qtd, dias):
    data_inicio = (hoje_brasilia - pd.Timedelta(days=dias)).strftime('%Y-%m-%d')
    try:
        dados = yf.download(ticker, start=data_inicio, progress=False, auto_adjust=True)
        preco_inicio = float(dados['Close'].dropna().iloc[0].squeeze())
        preco_hoje = float(dados['Close'].dropna().iloc[-1].squeeze())
    except Exception:
        print(f"[AVISO] Não foi possível obter {ticker} via yfinance. Tentando CoinGecko...")
        
        try:
            preco_inicio = get_crypto_price_brl_on_date_cached(ticker, data_inicio)
            print(data_inicio, preco_inicio)
            time.sleep(10)
            preco_hoje = get_crypto_price_brl(ticker)
            print(preco_hoje)
            time.sleep(10)
        except Exception as cg_err:
            print(f"[ERRO] Falha também com CoinGecko para {ticker}: {cg_err}")
            return None, None, None

    valor_inicial = preco_inicio * qtd
    valor_final = preco_hoje * qtd
    rentabilidade = (valor_final / valor_inicial - 1) * 100

    return valor_inicial, valor_final, rentabilidade

# Calcula tabela completa de rentabilidades
def calcular_tabela_rentabilidades(wallet):
    br_tz = pytz.timezone("America/Sao_Paulo")
    inicio_do_ano = br_tz.localize(datetime(hoje_brasilia.year, 1, 1))
    
    periodos = {
        "Rent. Diária (%)": 1,
        "Rent. Semanal (%)": 7,
        "Rent. Mensal (%)": 30,
        "Rent. YTD (%)": (hoje_brasilia - inicio_do_ano).days
    }

    resultado = []

    for i, row in wallet.iterrows():
        ticker = row['Ticker']
        qtd = row['Quantidade']
        linha = {"Ticker": ticker}

        for nome_periodo, dias in periodos.items():
            valor_ini, valor_fim, rent = calcular_rentabilidade_ativo(ticker, qtd, dias)
            linha[nome_periodo] = round(rent, 2) if rent is not None else None

        resultado.append(linha)

    df_resultado = pd.DataFrame(resultado)

    # Adiciona linha final com rentabilidade total da carteira ponderada
    total_row = {"Ticker": "Total"}
    for nome_periodo, dias in periodos.items():
        total_ini = 0
        total_fim = 0
        for i, row in wallet.iterrows():
            ticker = row['Ticker']
            qtd = row['Quantidade']
            valor_ini, valor_fim, _ = calcular_rentabilidade_ativo(ticker, qtd, dias)
            if valor_ini is not None:
                total_ini += valor_ini
                total_fim += valor_fim
        rent_total = (total_fim / total_ini - 1) * 100 if total_ini > 0 else None
        total_row[nome_periodo] = round(rent_total, 2) if rent_total is not None else None

    df_resultado = pd.concat([df_resultado, pd.DataFrame([total_row])], ignore_index=True)
    return df_resultado

profitability = calcular_tabela_rentabilidades(wallet)
profitability


1 Failed download:
['BITCOIN']: YFTzMissingError('possibly delisted; no timezone found')


[AVISO] Não foi possível obter bitcoin via yfinance. Tentando CoinGecko...
2025-07-03 590590.2588453247
592926.0077387808



1 Failed download:
['BITCOIN']: YFTzMissingError('possibly delisted; no timezone found')


[AVISO] Não foi possível obter bitcoin via yfinance. Tentando CoinGecko...
2025-06-27 586122.6110893537
592926.0077387808



1 Failed download:
['BITCOIN']: YFTzMissingError('possibly delisted; no timezone found')


...


1 Failed download:
['BITCOIN']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['BITCOIN']: YFTzMissingError('possibly delisted; no timezone found')


[AVISO] Não foi possível obter bitcoin via yfinance. Tentando CoinGecko...
[ERRO] Falha também com CoinGecko para bitcoin: Erro ao buscar histórico de bitcoin em 2025-06-27 (status 429)
[AVISO] Não foi possível obter bitcoin via yfinance. Tentando CoinGecko...



1 Failed download:
['BITCOIN']: YFTzMissingError('possibly delisted; no timezone found')


[ERRO] Falha também com CoinGecko para bitcoin: Erro ao buscar histórico de bitcoin em 2025-06-04 (status 429)
[AVISO] Não foi possível obter bitcoin via yfinance. Tentando CoinGecko...
[ERRO] Falha também com CoinGecko para bitcoin: Erro ao buscar histórico de bitcoin em 2025-01-01 (status 429)


,Ticker,Rent. Diária (%),Rent. Semanal (%),Rent. Mensal (%),Rent. YTD (%)
0,bitcoin,0.4,1.16,-0.28,2.52
1,Total,0.4,NaN,NaN,NaN
